# Periodic Boundary Conditions - Dynamics 

## Overview

In this session, we will learn how to compute dynamical quantities from an MD simulation. 

### Questions

- How do I compute dynamic quantities like the diffusion coefficient from a MD simulation?

### Objectives

- Calculate the mean-squared displacement of monomers, monomers relative to the center of mass of a polymer, and the center of mass MSD of a polymer.
- Extract the average diffusion coefficent from the measured MSD for a simple polymer model.
- Show that the simple polymer model agrees with scaling expectations (Rouse model).

## Boilerplate code

In [ ]:
import matplotlib
import numpy
import scipy.stats
import gsd, gsd.hoomd
import freud
import math
import itertools
import hoomd
import fresnel

%matplotlib inline
matplotlib.style.use("ggplot")
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

def render(frame):
    scene = fresnel.Scene()
    geometry = fresnel.geometry.Sphere(scene, N=frame.particles.N)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]), roughness=0.5)
    geometry.position[:] = frame.particles.position
    geometry.outline_width = 0.01
    box = fresnel.geometry.Box(scene, frame.configuration.box,box_radius=0.01)
    L = frame.configuration.box[0]
    scene.camera = fresnel.camera.Perspective(position=(L*1.8, L*1.8, L * 2.2), look_at=(0, 0, 0), up=(0, 1, 0), height=0.28)

    if frame.bonds.N>0:
        geometry.radius[:] = [0.2]*frame.particles.N

        all_bonds = numpy.stack(
        [
            frame.particles.position[frame.bonds.group[:, 0]],
            frame.particles.position[frame.bonds.group[:, 1]],
        ],
        axis=1,
        )
        # Use a distance cutoff (L/2) to filter bonds that span the periodic boundary
        bond_distances = numpy.linalg.norm(all_bonds[:,0,:]-all_bonds[:,1,:], axis=1)
        L = frame.configuration.box[0]
        bond_indices = numpy.where(bond_distances < L/2)[0]
        filtered_bonds = all_bonds[bond_indices, :, :]

        bonds = fresnel.geometry.Cylinder(scene, N=len(filtered_bonds))
        bonds.material = fresnel.material.Material(roughness=0.5)
        bonds.outline_width = 0.05

        bonds.points[:] = filtered_bonds
        bonds.radius[:] = [0.1]*len(filtered_bonds)
        bonds.material.primitive_color_mix = 1.0
        bonds.color[:] = fresnel.color.linear([0.8, 0.8, 0.8])

    return fresnel.preview(scene)

## Background

We will largely follow [Mussawisade, K., M. Ripoll, R. G. Winkler, and G. Gompper. "Dynamics of polymers in a particle-based mesoscopic solvent." The Journal of chemical physics 123, no. 14 (2005)](http://doi.org/10.1063/1.2041527) for our model/parameter choices. In this section, we will implement and run a Gaussian Chain simulation with no pair interactions. This will model a so-called "ideal polymer".

## Simulation 

### Initial Snapshot 

Write a function that initializes `num_pol` with length `num_mon` each. Initialize them as straight rods inside the box. Feel free to ajust the box size to be large enough to fit the polymers.

Hints:
1. Use `numpy.linspace` to make an array going from `-mon_num/2` to `mun_nom/2+1`. How many steps do you need to make a polymer of length `num_mon`?
2. then use `numpy.hstack` to create a position array of the correct shape. What *is* the correct shape? You can always print the shape of an numpy array by using `print(array.shape)` and check your work! 

In [ ]:
def initialize_snapshot(num_pol,num_mon):

    position_x = numpy.linspace(...)
    position = numpy.hstack((...))
    positions = []
    for i in range(num_pol):
        # offset to shift current polymer number i a little off its neighbors
        offset = [0,-num_mon/2+i*1.1,0]
        positions.append(position + offset)

    positions = numpy.asarray(positions).reshape(-1,3)

    bonds = numpy.vstack((numpy.arange(num_pol*num_mon-1),numpy.arange(1,num_pol*num_mon))).T
    bonds = bonds[bonds[:,1] % num_mon !=0 ]

    frame = gsd.hoomd.Frame()
    frame.particles.types = ['A']
    frame.particles.N = ...
    frame.particles.position = ...
    frame.bonds.N = ...
    frame.bonds.group = ...
    frame.bonds.types = ['b']
    L = 2*numpy.max(numpy.abs(position))
    frame.configuration.box = [L, L, L, 0, 0, 0]

    return frame


In [ ]:
test = initialize_snapshot(10,20)
render(test)

This is one simple way to set up a initial configuration. For any actual workflow, it might be worthwile to improve this initial snapshot setup to get closer to a correct configuration, i.e for example one could implement a random walk or a self avoiding random walk. 

## Simulation Setup 

The `num_mon` monomers in a Guassian chain are connected to form a polymer of length `mum_mon` by a *harmonic* bond potential. The spring constant is $\frac{3 k_{B} T}{2 b^{2}}$ and  with zero mean bond length. Here, $b$ is the root-mean-square bond length and $k_BT$ is the temperature. We rename the the root-mean-square bond length variable to $b$ (from $l$ used in the original paper) to avoid conflict with variable names used later.
*Note* that the factor of $1/2$ is already in hoomd-blue's definition of the [harmonic bond potential](https://hoomd-blue.readthedocs.io/en/stable/hoomd/md/bond/harmonic.html). 

Since the chains are non-interacting, i.e., they have no excluded volume, we might as well put some number of polymers, `num_pol`, into the simulation box to improve the statistics of the measurnments done later. If the chains were interacting, we would need to think about the effects of polymer concentrations. 

In [ ]:
kT = 1.0
l = 1.0
harmonic = hoomd.md.bond.Harmonic()
harmonic.params["b"] = dict(r0=0, k=3*kT/l**2)

Now we can set up the rest of the simulation exactly like we have done for previous Langevin simulations.

**Note** that we write the images into the GSD files here by using `dynamic` keyword with both `property` and `momentum`. 

In [ ]:
timestep = 0.01
gamma = 0.1

num_pol = 10
num_mon = 80

frame = initialize_snapshot(num_pol,num_mon)

# add code
integator = ...

integrator.forces.append(harmonic)

# add code
simulation = ...
simulation.operations.integrator = ...
simulation.create_state_from_snapshot(...)

# Equilibrate
simulation.run(100_000)

gsd_out = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(1_000),
    mode='wb',
    dynamic=['property','momentum'],
    filename='run_len_%s_pol_%s.gsd'%(num_mon,num_pol))

simulation.operations.writers.append(gsd_out)
# run
simulation.run(5_000_000)

gsd_out.flush()
render(simulation.state.get_snapshot())

While the simulation is running, look up what the scaling relationship for the [diffusion coefficent](https://en.wikipedia.org/wiki/Rouse_model) is expected to be for a simple polymer model without excluded volume interactions or hydrodynamics. Look up the **Rouse** and the **Zimm** model. 

To check if our simulation has run properly, we can check the bond lengths. The bond length distribution should follow a Gaussian distribution (as the name of the model implies). 

Hints: 
1. We can use `numpy.linalg.norm()` to compute the length of a vector. What axis to we need to indicate? Hint: check the shapes of your arrays.
2. Compute the bond vector by using the unwrapped positions. Rember that the first index in each bond is given by `frame.bonds.group[:, 0]` and the second one is `frame.bonds.group[:, 1]`. 

In [ ]:
trajectory =  gsd.hoomd.open('run_len_80_pol_10.gsd','r')

all_bonds = []
for frame in trajectory:
    # add code - this is the same as we have done for the Rg exercise!
    unwrapped_positions = ...
    bond_vectors =
    bonds_lengths =  ...
    all_bonds.append(bonds_lengths)

fig,ax = matplotlib.pyplot.subplots(1,1)

hist , edges = numpy.histogram(...)

# add code to calculate the centers of each bin
center = ...
ax.plot(center,hist)
ax.set_xlabel("bond length")
ax.set_ylabel("count")
matplotlib.pyplot.show()

## Analyze MSD 

To analyze the MSD, we will write a function that takes a filename for gsd trajectory, opens the trajectory and computes the MSD using [freud](https://freud.readthedocs.io/en/latest/modules/msd.html). We will compute the monomer MSD ($g_1$), the relative monomer to center of mass MSD ($g_2$), and the center of mass MSD ($g_3$). We expect both the monomer and the com MSD to eventually scale linearly with time, $MSD \sim t^1$. The relative monomer to center of mass diffusion will plateau out at long times. 

**Write a function that calculates all three MSDs.**

You will need to iterate over the trajectory, find a way to separate the positions for the different polymers (see last lecture!), calculate the center of mass of each polymer, and then save the unwrapped positions, center of mass, and the unwrapped positions relative (minus) the center of masses in arrays of appropriate shape. 

Note that we want to substract a *global* center of mass drift, i.e entire simulation drift. This drift should be minimal if the thermostat is working correctly. Especially for systems with only a few particles, there still can be some effect, simply because there are not enough velocities to ensure that the mean of all velocities is exactly zero. In any case, it does not hurt to substract the global center of mass from the positions once they are unwrapped. 

Once you have three arrays, one with unwrapped positions `all_unwrapped_positions`, one with relative unwraped positions `all_unwrapped_coms_particles`, and one with center of masses `all_unwrapped_coms`, we can use freud to compute the MSD in the next step. 

Hints: 
1. When we compute these arrays, feel free to print out their shapes at various steps to debug. 
2. We need to use `numpy.repeat` to adjust the size of the `unwrapped_coms` and the `unwrapped_positions` array to be able to substract them from each other. `numpy.repeat` will take an array `[1,2,3,..]` and turn it into `[1,1,1,..2,2,2,..,3,3,3,..]`. How often do we need to repeat and over which axis? 


In [ ]:
def analyze_msd(filename,timestep):
    trajectory =  gsd.hoomd.open(filename,'r')

    all_unwrapped_positions = []
    all_unwrapped_coms = []
    all_unwrapped_coms_particles = []
    times = []

    # Hint: this is the same as for the Rg exercise!
    box  = ...
    bonds = ...
    polymer_delimiters = ...
    time0 = trajectory[0].configuration.step
    for frame in trajectory:

        # unwrap by adding frame*box to the positions:
        unwrapped_positions = ...
        # substract global com
        unwrapped_positions = unwrapped_positions -numpy.mean(unwrapped_positions,axis=0)
        all_unwrapped_positions.append(...)

        # split polymers, calculate com for each, append to proper arrays
        polymers = numpy.split(...)
        unwrapped_coms = []
        for p in polymers:
            com = numpy.mean(...)
            unwrapped_coms.append(com)

        # currently our com array and our particles (unwrapped positions) array have very different shapes.
        # we need to fix that so we can substract them to compute positions - com.
        unwrapped_coms_tiled = numpy.repeat(...)
        diff = unwrapped_positions - unwrapped_coms_tiled
        all_unwrapped_coms_particles.append(diff)

        all_unwrapped_coms.append(...)
        times.append(frame.configuration.step-time0)

    all_unwrapped_positions = numpy.array(all_unwrapped_positions)  # shape = (nframes,nparticles,3)
    all_unwrapped_coms = numpy.array(all_unwrapped_coms)            # shape = (nframes,npolymers,3)
    all_unwrapped_coms_particles = numpy.array(all_unwrapped_coms_particles)  # shape = (nframes,nparticles,3)

    # add MSD msd1 calcuation using freud

    msd_calculator = freud.msd.MSD()
    # put postions here
    msd_calculator.compute(...)
    # average MSD over all particles (or com, depending)
    result = msd_calculator.particle_msd
    msd1 = numpy.nanmean(result,axis=1)

    # repeat code above to comute msd2 and msd3
    ...


    lagtimes = numpy.asarray(times)*timestep

    return lagtimes, msd1, msd2, msd3

Now we can plot our results for one chain length, let's say $N=80$ and check against our expectations. We will also take the derivative `numpy.gradient` to compute $D(t)$.

In [ ]:
fig,ax = matplotlib.pyplot.subplots(2,1)

# plot the 3 msds in one panel, and D(t) in the second panel.
lagtimes, msd1, msd2, msd3 = analyze_msd("run_len_80_pol_10.gsd",0.01)

ax[0].plot(lagtimes, msd1,label='g1 = monomer msd')
ax[0].plot(lagtimes, msd2,label='g2 = monomer-com msd')
ax[0].plot(lagtimes, msd3,label='g3 = com msd')

gradient = numpy.gradient(msd3, lagtimes)
ax[1].plot(lagtimes,gradient/(6.0),label='D(t)=1/6 d/dt g3')


ax[0].set_ylabel("$MSD$")
ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].legend()
ax[1].legend()

ax[1].set_xlabel(r"$time$")
ax[1].set_ylabel("$D$")

matplotlib.pyplot.show()

Now, repeat the simulation we have done above for a few values of $N$ in the range of 20 to 80. Keep the number of polymers `num_pol` constant at 10.

In [ ]:
# copy code from above OR modify the code above to change chain length
...

After that, let's determine the $MSD$, $D(t)$ and $\langle D\rangle$ of each chain length:

In [ ]:
fig,ax = matplotlib.pyplot.subplots(2,1)

# we will take the average of D(t) only in the range over which it appears to be constant
fit_range = [10,30000]

# add your chain lengths
Ns = [....]
all_av_D = []

for N in Ns:
    lagtimes, msd1, msd2, msd3 = analyze_msd(...)
    D = numpy.gradient(...)/...
    ax[0].plot(lagtimes, msd3,label='N=%s'%N)
    ax[1].plot(lagtimes,D)
    diffusion_coefficient = numpy.mean(D[(lagtimes>fit_range[0]) & (lagtimes<fit_range[1])])

    all_av_D.append(diffusion_coefficient)

# plot results
ax[0].set_ylabel("$MSD$")
ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].legend()
# adjust if needed
ax[1].set_ylim(-0.05,0.2)
ax[1].set_xlabel(r"$time$")
ax[1].set_ylabel("$D$")

matplotlib.pyplot.show()

For comparison, load the file `gaussian-D-Rg.txt`, which contains calculated diffusion coefficents (and $R_g$ in column 3) for $N=10$ to $100$ and plot your data with those diffusion coefficients. Do they follow the expected scaling law? 

In [ ]:
fig,ax = matplotlib.pyplot.subplots(1,1)

x = numpy.arange(10,100,1)
# load file
data = numpy.genfromtxt("gaussian-D-Rg.txt")
# add scaling fit
ax.plot(x,...,c='blue')
ax.scatter(data[:,0], data[:,1])
ax.scatter(Ns, all_av_D)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r"$N$")
ax.set_ylabel(r"$\langle D \rangle$")
matplotlib.pyplot.show()


## Additions 

1. How would you decide on the `fit_range` for the diffusion coefficient? *Hint:* Is there a way to estimate the uncertainty of $D(t)$?
2. What happens if we add pair interactions to our simulation? What solvent qualities can we model? Do the results still agree with the Rouse model expectations?
3. What happens if we change the temperature `kT`$=k_{\mathrm {B}}T$ or the friction coefficient `gamma` in our simulations? Do the results still agree with the Rouse model expectations?